In [1357]:
import numpy as np
from collections import defaultdict, namedtuple

In [1327]:
# https://arxiv.org/pdf/cs/0011047.pdf
# https://medium.com/optima-blog/solving-sudoku-fast-702912c13307

In [1328]:
A = [[0, 0, 1, 0, 1, 1, 0],
     [1, 0, 0, 1, 0, 0, 1],
     [0, 1, 1, 0, 0, 1, 0],
     [1, 0, 0, 1, 0, 0, 0],
     [0, 1, 0, 0, 0, 0, 1],
     [0, 0, 0, 1, 1, 0, 1]]
A = np.array(A)

In [1362]:
sudoku = np.array([[0, 0, 0, 0, 0, 4, 0, 9, 0],
                   [8, 0, 2, 9, 7, 0, 0, 0, 0],
                   [9, 0, 1, 2, 0, 0, 3, 0, 0],
                   [0, 0, 0, 0, 4, 9, 1, 5, 7],
                   [0, 1, 3, 0, 5, 0, 9, 2, 0],
                   [5, 7, 9, 1, 2, 0, 0, 0, 0],
                   [0, 0, 7, 0, 0, 2, 6, 0, 3],
                   [0, 0, 0, 0, 3, 8, 2, 0, 5],
                   [0, 2, 0, 5, 0, 0, 0, 0, 0]])
# sudoku = np.array([[7, 3, 5, 6, 1, 4, 8, 9, 2],
#                    [8, 4, 2, 9, 7, 3, 5, 6, 1],
#                    [9, 6, 1, 2, 8, 5, 3, 7, 4],
#                    [2, 8, 6, 3, 4, 9, 1, 5, 7],
#                    [4, 1, 3, 8, 5, 7, 9, 2, 6],
#                    [5, 7, 9, 1, 2, 6, 4, 3, 8],
#                    [1, 5, 7, 4, 9, 2, 6, 8, 3],
#                    [6, 9, 4, 7, 3, 8, 2, 1, 5],
#                    [3, 2, 8, 5, 6, 1, 7, 4, 9]])
# sudoku = [[0, 6, 0, 3, 0, 0, 8, 0, 4],
#           [5, 3, 7, 0, 9, 0, 0, 0, 0],
#           [0, 4, 0, 0, 0, 6, 3, 0, 7],
#           [0, 9, 0, 0, 5, 1, 2, 3, 8],
#           [0, 0, 0, 0, 0, 0, 0, 0, 0],
#           [7, 1, 3, 6, 2, 0, 0, 4, 0],
#           [3, 0, 6, 4, 0, 0, 0, 1, 0],
#           [0, 0, 0, 0, 6, 0, 5, 2, 3],
#           [1, 0, 2, 0, 0, 9, 0, 8, 0]]
# sudoku = np.array(sudoku)

# sudoku = [[0, 0, 0, 0, 0, 0, 0, 1, 2],
#           [0, 0, 0, 0, 0, 0, 0, 0, 3],
#           [0, 0, 2, 3, 0, 0, 4, 0, 0],
#           [0, 0, 1, 8, 0, 0, 0, 0, 5],
#           [0, 6, 0, 0, 7, 0, 8, 0, 0],
#           [0, 0, 0, 0, 0, 9, 0, 0, 0],
#           [0, 0, 8, 5, 0, 0, 0, 0, 0],
#           [9, 0, 0, 0, 4, 0, 5, 0, 0],
#           [4, 7, 0, 0, 0, 6, 0, 0, 0]]
# sudoku = np.array(sudoku)

In [1330]:
# Data object, x.
class X:
    def __init__(self, column=None, value=None, row=None):
        self.up = self.down = self.right = self.left = self
        self.column = column # Points to the column object at the head of the relevant column.
        self.value = value
        self.row = row
    def __str__(self):
        return f'{self.column.name}:{self.value}'

# Column object, y.
class Y(X):
    def __init__(self, value=None, row=None, name='', size=0):
        self.size = size # The number of 1s in the column.
        self.name = name # Symbolic identifier for printing the answers.
        
        X.__init__(self, self, value, row)

In [1331]:
def init_root(A, labels):
    rows, cols = A.shape
    # The root header links all the column headers.
    root = Y(name='root', size=float('inf'))

    # Prepare the header list.
    curr = root
    for col in range(cols):
        curr.right = Y(name=labels[col])
        curr.right.left = curr
        curr = curr.right
        
    # Don't forget to link it back to the root - it's circular in nature.
    curr.right = root
    curr.right.left = curr

    # Simple pointer to help us find the column at constant time.
    header = {}
    curr = root.right
    while curr != root:
        header[curr.name] = curr
        curr = curr.right
    return root, header

In [1365]:
def init_toroidal(A):
    rows, cols = A.shape
    # Use a human-readable label.
    labels = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    
    # In case we have a lot of columns...
    if cols >= len(labels):
        labels = list(map(str, range(cols)))

    root, header = init_root(A, labels)
    for i, row in enumerate(A):
        prev = None
        left = None
        for j, col in enumerate(row):
            if col != 1: continue

            head = header[labels[j]]
            head.size += 1

            curr = head.up
            curr.down = X(column=head, 
                          value=col, 
                          row=i)
            curr.down.up = curr

            curr = curr.down
            curr.down = curr.column
            curr.down.up = curr

            if prev is None:
                prev = curr
                prev.right = curr
                prev.right.left = prev
                left = curr
            else:
                prev.right = curr
                prev.right.left = prev
                prev = curr
        # Happens when the column does not have any 1s.
        if prev is not None:
            prev.right = left
            prev.right.left = prev
    return root

In [1341]:
def pprint():
    keys = header.keys()
    for key in keys:
        node = header[key]
        print(node.size)
        curr = node.down
        while curr != node:
            print(curr)
            curr = curr.down
        right = node.down.right

        while right != node.down:
            print(right, end=' ')
            right = right.right

In [1342]:
# Choose a column object.
def column_object_heuristic(root):
    '''
    Attempt to select the column object with the smallest size.
    '''
    curr = root.column.right
    c = root
    while curr != root:
        if curr.size < c.size:
            c = curr
        curr = curr.right
    return c

In [1343]:
# column_object_heuristic(root).name

In [1344]:
def cover(col):
    col = col.column
    col.right.left = col.left
    col.left.right = col.right
    i = col.down
    while i != col:
        j = i.right
        while j != i:
            j.up.down = j.down
            j.down.up = j.up
            j.column.size -= 1    
            j = j.right
        i = i.down

In [1345]:
def uncover(col):
    col = col.column
    i = col.up
    while i != col:
        j = i.left
        while j != i:
            j.column.size += 1
            j.up.down = j
            j.down.up = j
            j = j.left
        i = i.up
    col.right.left = col
    col.left.right = col

In [1346]:
def search(root, k=0, solution=None):
    if solution is None:
        solution = []
    if root.right == root:
        return solution[:]
    
    col = column_object_heuristic(root)
    cover(col)

    r = col.down
    while r != col:
        o_k = r
        solution.append(o_k.row)
        
        j = r.right
        while j != r:
            cover(j)
            j = j.right
        
        result = search(root, k+1, solution)
        if result: return result
        solution.remove(o_k.row)

        r = o_k
        col = r.column
        
        j = r.left
        while j != r:
            uncover(j)
            j = j.left
        r = r.down
    uncover(col)

In [1366]:
torodial = init_toroidal(A)
result = search(torodial)
result

[3, 0, 4]

In [1367]:
A[result, :].sum(axis=0)

array([1, 1, 1, 1, 1, 1, 1])

In [1358]:
Metadata = namedtuple('Metadata', 'row col val')

def constraint_index(row, col, val):
    '''
    Returns the index of the unique constraint in a 9x9 row.
    For all four constraints below, we will construct a 4x9x9
    row and mark the index.
    '''
    r = row // 3 * 3 # Gives the current grid row.
    c = col // 3     # Gives the current grid col.
    grid = r + c     # The grid number from top to bottom, left to right.
    # Grid:
    # 012
    # 345
    # 678
    return (row * 9 + col,         # Each cell can only have one number.
            row * 9 + val - 1,     # For each row, a number can only appear once.
            col * 9 + val - 1,     # For each column, a number can only appear once.
            grid * 9 + val - 1)    # For each region, a number can only appear once.

def build_constraints(sudoku):
    '''
    Build the constraint matrix for all the existing values in the sudoku board,
    as well as the ones that we have not seen yet. Dancing links will try
    to find all the possible combinations.
    '''
    
    # The actual size of the result is 9 * 9 * 9 for every combination of numbers in each row, col and grid.
    # If we have a partial solution (filled numbers in the sudoku), we can eliminate most of them.
    result, metadata = [], []
    for row in range(0, 9):
        for col in range(0, 9):
            val = sudoku[row, col]
            if val == 0:
                # The cell is empty. We need to fill it with all the values that are not inside yet.
                # To do so, find the values that are not in the current row yet.
                # ! Numbers must be in range 1-9, not index 0.
                values = [i for i in range(1, 10) 
                          if i not in sudoku[row]]
            else:
                values = [val]
            for val in values:
                cell_c, row_c, col_c, grid_c = constraint_index(row, col, val)
                mat = np.zeros(4*9*9) # Each constraints have 81 indices.
                mat[cell_c + 0 * 81] = 1
                mat[row_c + 1 * 81] = 1
                mat[col_c + 2 * 81] = 1
                mat[grid_c + 3 * 81] = 1
                result.append(mat)
                metadata.append(Metadata(row=row, col=col, val=val))
    return np.array(result), metadata

In [1369]:
constraints, metadata = build_constraints(sudoku)
torodial = init_toroidal(constraints)
rows = search(torodial)

# Draw back the solution.
solution = np.zeros((9, 9))
for row in rows:
    cell = metadata[row]
    solution[cell.row, cell.col] = cell.val
solution

array([[7., 3., 5., 6., 1., 4., 8., 9., 2.],
       [8., 4., 2., 9., 7., 3., 5., 6., 1.],
       [9., 6., 1., 2., 8., 5., 3., 7., 4.],
       [2., 8., 6., 3., 4., 9., 1., 5., 7.],
       [4., 1., 3., 8., 5., 7., 9., 2., 6.],
       [5., 7., 9., 1., 2., 6., 4., 3., 8.],
       [1., 5., 7., 4., 9., 2., 6., 8., 3.],
       [6., 9., 4., 7., 3., 8., 2., 1., 5.],
       [3., 2., 8., 5., 6., 1., 7., 4., 9.]])